In [1]:
import numpy as np
import random
from copy import copy

from simulation_helpers import generateAdjacenyMatrix
from optimal_policy import V

In [68]:
n = 100
L = 5
p = 0.02
q = 0
h = 10
np.random.choice?

In [ ]:
def selectRandom(y, Q, A, L):
    '''
    Heuristic that tests uninfected people randomly
    '''
    L = min(L, Q.sum())
    
    tests = np.zeros(len(Q))
    tests[np.random.choice(np.nonzero(Q)[0], L, replace = False)] = 1
    
    return tests

In [ ]:
def highRisk(y, Q, A, L):
    '''
    Heuristic that tests those closest to those removed from the graph
    '''
    L = min(L, Q.sum())
    
    #Computee number of 'removed' people everyone is connected to
    R = np.matmul(A, 1-Q)
    R[1-Q] = 0 #If person is removed set 'removed' connections to 0
    
    tests = np.zeros(len(Q))
    tests[np.argsort(R)[::-1][:L]] = 1 #Test the top L most connected to removed nodes

    return tests

In [ ]:
def highConnection(y, Q, A, L):
    '''
    Heuristic that tests those that are most connected on the graph
    '''
    L = min(L, Q.sum())
    
    #Compute number of people everyone is connected to
    R = np.matmul(A, np.ones(len(Q)))
    R[1-Q] = 0 #If person is removed set 'number of connections to 0
    
    tests = np.zeros(len(Q))
    tests[np.argsort(R)[::-1][:L]] = 1 #Test the top L most connected to removed nodes

    return tests

In [ ]:
def highBeliefRisk(y, Q, A, L, consider_removed = False):
    '''
    Heuristic that tests those that are most connected to the people most likely to be sick
    '''
    L = min(L, Q.sum())
    
    #Adjust Y to deal with removed nodes
    y_sample = y.copy()
    y_sample[1-Q] = 1 if consider_removed else 0 #Takes advantage of fact removed nodes = 1

    #Compute number of people everyone is connected to
    R = np.matmul(A+np.eye(A.shape[0]), y_sample) #Add Identity matrix so that we include belief on person themself
    R[1-Q] = 0 #If person is removed set risk to 0
    
    tests = np.zeros(len(Q))
    tests[np.argsort(R)[::-1][:L]] = 1 #Test the top L most connected to removed nodes

    return tests

In [ ]:
def highBelief(y, Q, A, L):
    '''
    Heuristic that tests the L individuals with highest beliefs
    '''
    L = min(L, Q.sum())
    
    #Adjust y to remove nodes outside of graph
    y_sample = y.copy()
    y_sample[1-Q] = 0
    
    tests = np.zeros(len(Q))
    tests[np.argsort(y_sample)[::-1][:L]] = 1 #Test the top L nodes with highest beliefs
    return tests

In [ ]:
def sampleBelief(y,Q,A,L, softmax = True):
    '''
    Heuristic that samples who to test by building a (softmax) distirbution over nodes
    '''
    L = min(L, Q.sum())
    
    #Adjust y to remove nodes outside of graph
    y_sample = y.copy()
    y_sample[1-Q] = 0
    
    #Build a distribution over nodes based on belief
    dist = np.exp(y_sample)/np.exp(y_sample).sum() if softmax else y/y.sum()
    
    tests = np.zeros(len(Q))
    tests[np.random.choice(np.arange(len(Q)), L, p = dist, replace = False)] = 1 #Test L nodes sampled from distribution
    return tests

In [6]:
def removeNodes(y, Q, test, q):
    
    a = np.empty_like (y)
    a[:] = y
    
    # amounce the infected, announce == 0 if that person decides to announce
    announce = np.random.binomial(1,q,len(a))    
    announce = np.multiply(a,announce)  
    announce = 1 - announce
    
    # remove the person who has announced from the graph
    Q[announce == 0] = announce[announce == 0]
    

    # test result according to the test we selected
    testResult = np.multiply(y,test)
    testResult = 1-testResult
    
    # remove the people that are tested infected 
    Q[testResult == 0] = testResult[testResult == 0]
    
    return Q

In [ ]:
def runSimulation(A, p, q, L, H, policy):
    n = A.shape[0]
    y = np.ones(n)*1/(n) #uniform prior
    Q = np.ones(n) #everyone starts in graph
    
    #randomly choose first infected individual
    infected = np.zeros(n)
    infected[np.random.choice(np.arange(n),1)] = 1
    
    for h in range(H):
        at_risk = np.matmul(A[Q,:][:,Q], infected[Q]) > 0
        infected[Q][at_risk] = np.clip(infected[Q][at_risk] + np.random.binomial(1, p, size=at_risk.sum()),0,1)
        tests = policy(y,Q,A,L)
        
        Q[infected] = 
        

In [133]:
n=10
infected = np.zeros(n)
infected[np.random.choice(np.arange(n),1)] = 1
A = generateAdjacenyMatrix(n,0) + np.eye(n)
Q = np.ones(n).astype(np.bool)
at_risk = np.matmul(A[Q,:][:,Q], infected[Q]) > 0
print(infected[Q,])
#infected[Q[at_risk]] = np.clip(infected[Q][at_risk] + np.random.binomial(1, 1, size=at_risk.sum()),0,1)
#infected
#np.clip(infected[Q][at_risk] + np.random.binomial(1, 1, size=at_risk.sum()),0,1)
#np.clip(np.zeros(10) + np.random.binomial(1, 0.2, 10),0,1)

IndexError: too many indices for array

In [7]:
def infection(y, Q, p, A):
    R = np.zeros(len(Q))
    
    R[Q] = np.matmul(A[Q,:][:,Q], y[Q])

    infection = np.zeros(len(Q))

    for i in range(len(R)):
        source = int(R[i])

        if source > 0:
            v = np.random.binomial(1, p, source)
            if v.sum() > 0:
                infection[i] = 1
    
    return np.clip(y + infection, 0, 1)

In [8]:
def simulationRandom(Q, h, p, q, L, A, n):
    numInf = np.zeros(h)
    
    y = np.zeros(n)
    
    # randomly initialize an infected person
    start = random.randint(0,n-1) 
    y[start] = 1
    
    done = False 
    
    
    for n in range(h): 
            # done if no healthy ppl connected to someone sick or nobody healthy on graph
            R = np.zeros(len(Q))    
            R[Q] = np.matmul(A[Q,:][:,Q], y[Q])            
            done = (max(R) == 0) or (min(y[Q]) == 1)
            
            # else proceed 
            if not done:
                test = selectRandom(Q,L)
                Q = removeNodes(y, Q, test, q)
                y = infection(y, Q, p, A)            
                Q[start] = 0
                
                
            # count the # of infected on the graph
            Q = Q.astype(np.bool)
            inf = y[Q]  
            numInf[n] = int(inf.sum())

        
    return numInf

In [9]:
def simulationRisky(Q, h, p, q, L, A, n):
    
    numInf = np.zeros(h)
    
    y = np.zeros(n)
    start = random.randint(0,n-1) 
    y[start] = 1
    removed = y
    
    for n in range(h):
        # done if no healthy ppl connected to someone sick or nobody healthy on graph
        R = np.zeros(len(Q))    
        R[Q] = np.matmul(A[Q,:][:,Q], y[Q])            
        done = (max(R) == 0) or (min(y[Q]) == 1)
            
        # else proceed 
        if not done:
         
            oldQ = Q.astype(int)
        
            test = highRisk(y, Q, A, removed, L)
        
            Q = removeNodes(y, Q, test, q)
        
            newQ = Q.astype(int)

            removed = oldQ - newQ if n >= 1 else y

        
            y = infection(y, Q, p, A)

        
            if Q[start] == 1:
                Q[start] = 0
        
        # count the # of infected on the graph
        Q = Q.astype(np.bool)
        inf = y[Q]               
        numInf[n] = int(inf.sum())
        
    return numInf

In [10]:
def resultRandom(h, p, q, L, n):
    result1 = np.zeros(h)
    for i in range(round):
        A = generateAdjacenyMatrix(n, 0.8)
        Q = np.ones(n).astype(np.bool)    
        a = simulationRandom(Q, h, p, q, L, A, n)
        result1 += a
        

    return result1 / float(round)

In [11]:
print resultRandom(h, p, q, L, n)

[ 1.   1.8  2.5  3.4  4.6  6.2  9.  11.  15.6 19.1]


In [12]:
def resultRisky(j, p, q, L, n):
    result2 = np.zeros(h)
    for i in range(round):
        A = generateAdjacenyMatrix(n, 0.8)
        Q = np.ones(n).astype(np.bool)
        b = simulationRisky(Q, h, p, q, L, A, n)
        result2 += b
        

    return result2 / float(round)

In [13]:
print resultRisky(h, p, q, L, n)

[ 0.6  0.7  1.2  1.6  2.4  4.   6.6 10.2 13.9 17.4]
